# How to stream messages from your graph

There are multiple different streaming modes.

- `values`: This streaming mode streams back values of the graph. This is the **full state of the graph** after each node is called.
- `updates`: This streaming mode streams back updates to the graph. This is the **update to the state of the graph** after each node is called.
- `messages`: This streaming mode streams back messages - both complete messages (at the end of a node) as well as **tokens** for any messages generated inside a node. This mode is primarily meant for powering chat applications.


This notebook covers `stream_mode="messages"`.

In order to use this mode, the state of the graph you are interacting with MUST have a messages key that is a list of messages.
Eg, the state should look something like:

```python
from typing import TypedDict, Annotated
from langgraph.graph import add_messages
from langchain_core.messages import AnyMessage

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
```

OR it should be an instance or subclass of `from langgraph.graph import MessageState` (`MessageState` is just a helper type hint equivalent to the above).

With `stream_mode="messages"` two things will be streamed back:

- It outputs messages produced by any chat model called inside (unless tagged in a special way)
- It outputs messages returned from nodes (to allow for nodes to return `ToolMessages` and the like

In [1]:
from langgraph_sdk import get_client

In [2]:
client = get_client()

In [3]:
assistant_id = "agent"

In [4]:
thread = await client.threads.create()
thread

{'thread_id': 'e1431c95-e241-4d1d-a252-27eceb1e5c86',
 'created_at': '2024-06-21T15:48:59.808924+00:00',
 'updated_at': '2024-06-21T15:48:59.808924+00:00',
 'metadata': {}}

In [5]:
runs = await client.runs.list(thread['thread_id'])
runs

[]

In [6]:
# Helper function for formatting messages

def format_tool_calls(tool_calls):
    if tool_calls:
        formatted_calls = []
        for call in tool_calls:
            formatted_calls.append(f"Tool Call ID: {call['id']}, Function: {call['name']}, Arguments: {call['args']}")
        return "\n".join(formatted_calls)
    return "No tool calls"

In [7]:
input = {"messages": [{"role": "user", "content": "what's the weather in sf"}]}
config = {"configurable": {"model_name": "openai"}}

async for event in client.runs.stream(thread['thread_id'], assistant_id, input=input, config=config, stream_mode='messages'):
    if event.event == 'metadata':
        print(f"Metadata: Run ID - {event.data['run_id']}")
        print("-" * 50)
    elif event.event == 'messages/partial':
        for data_item in event.data:
            if 'role' in data_item and data_item['role'] == 'user':
                print(f"Human: {data_item['content']}")
            else:
                tool_calls = data_item.get('tool_calls', [])
                invalid_tool_calls = data_item.get('invalid_tool_calls', [])
                content = data_item.get('content', "")
                response_metadata = data_item.get('response_metadata', {})

                if content:
                    print(f"AI: {content}")
                
                if tool_calls:
                    print("Tool Calls:")
                    print(format_tool_calls(tool_calls))
                
                if invalid_tool_calls:
                    print("Invalid Tool Calls:")
                    print(format_tool_calls(invalid_tool_calls))

                if response_metadata:
                    finish_reason = response_metadata.get('finish_reason', 'N/A')
                    print(f"Response Metadata: Finish Reason - {finish_reason}")
        print("-" * 50)
    

Metadata: Run ID - 1ef2fe5c-6a1d-6575-bc09-d7832711c17e
--------------------------------------------------
Invalid Tool Calls:
Tool Call ID: call_cg14F20jMBqWYrNgEkdWHwB3, Function: tavily_search_results_json, Arguments: 
--------------------------------------------------
Tool Calls:
Tool Call ID: call_cg14F20jMBqWYrNgEkdWHwB3, Function: tavily_search_results_json, Arguments: {}
--------------------------------------------------
Tool Calls:
Tool Call ID: call_cg14F20jMBqWYrNgEkdWHwB3, Function: tavily_search_results_json, Arguments: {}
--------------------------------------------------
Tool Calls:
Tool Call ID: call_cg14F20jMBqWYrNgEkdWHwB3, Function: tavily_search_results_json, Arguments: {'query': ''}
--------------------------------------------------
Tool Calls:
Tool Call ID: call_cg14F20jMBqWYrNgEkdWHwB3, Function: tavily_search_results_json, Arguments: {'query': 'current'}
--------------------------------------------------
Tool Calls:
Tool Call ID: call_cg14F20jMBqWYrNgEkdWHwB3, F